In [1]:
import os
import sys
from pathlib import Path
import logging
import time
import pandas as pd
import geopandas as gpd
import seaborn
import dask
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

sys.path.insert(0, '..')
import src.hotspot_utils as util 
import src.process_nearest_hotspots as nearest_process

In [2]:
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
_LOG = logging.getLogger(__name__)

In [3]:
# include n_workers equal or less than the number of core
client = Client(n_workers=15)
client

Client Scheduler: tcp://127.0.0.1:34719 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 15 Cores: 15 Memory: 66.57 GB


In [4]:
# This is the output directory outputs and itermediary files from this notebook examples will be stored.
outdir = Path("/home/jovyan/s3vt_dask/s3vtdata/workdir")

# Processing Parameter used in Sub-setting Spatial Extent and Temporal Range for Area of Interest
##### The FRP data from nasa, esa, eumetsat and landgate are merged, sub-setted and neareast hotspots csv files are generated based on the parameters in `processing_parameters`  
##### The parameter `chunks` in blocking FRP data to enable multi-processing. If you encounter memory issues then higher the number.
##### The parameter `compare_field` is the name of column that will be used to generate nearest hotspots, the data in the column should be of datetime type.

In [5]:
processing_parameters = {
    "nasa_frp": "s3://s3vtaustralia/nasa_hotspots_gdf.geojson",
    "esa_frp": "s3://s3vtaustralia/s3vt_hotspots.geojson",
    "eumetsat_frp": "s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson",
    "landgate_frp": "s3://s3vtaustralia/landgate_hotspots_gdf.geojson",
    "dea_frp": None,
    "lon_west": 147.0,
    "lat_south": -38.0,
    "lon_east": 154.0,
    "lat_north": -27.,
    "start_date": "2019-11-01",
    "end_date": "2020-10-08",
    "start_time": "21:00",
    "end_time": "3:00",
    "chunks": 500,
    "outdir": outdir,
    "compare_field": "solar_day",  # solar_day or solar_night
    "swath_config_file": Path("/home/jovyan/s3vt_dask/s3vtdata/configs/s3vtconfig.yaml"),
}

In [6]:
# This is to generate nearest .csv files. If .csv files already exists then skip this process. Takes around ~5-6 hours in this sandbox environment with 2-core and 16 GB RAM
nearest_hotspots_product_files = nearest_process.process_nearest_points(**processing_parameters)

2021-04-29 06:39:50,372: INFO: Processing FRP Hotspots Datasets
2021-04-29 06:39:50,389: INFO: Found credentials in environment variables.
2021-04-29 06:39:50,425: INFO: Fetching FRP datasets...
2021-04-29 06:39:50,426: INFO: s3://s3vtaustralia/nasa_hotspots_gdf.geojson exists: skipped download
2021-04-29 06:39:50,427: INFO: s3://s3vtaustralia/s3vt_hotspots.geojson exists: skipped download
2021-04-29 06:39:50,427: INFO: s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson exists: skipped download
2021-04-29 06:39:50,428: INFO: s3://s3vtaustralia/landgate_hotspots_gdf.geojson exists: skipped download
2021-04-29 06:39:50,428: INFO: dea Hotspots FRP  is None. excluding from analysis.
2021-04-29 06:39:50,429: INFO: Reading...
2021-04-29 06:39:50,429: INFO: reading and subsetting GeoDataFrame for nasa: /home/jovyan/s3vt_dask/s3vtdata/workdir/nasa_hotspots_gdf.geojson
2021-04-29 06:41:37,011: INFO: reading and subsetting GeoDataFrame for esa: /home/jovyan/s3vt_dask/s3vtdata/workdir/s3vt_hotspot

## Nearest Hotspots DataFrame merged from neareast hotspots csv files

In [7]:
# csv directory is where nearest hotspots csv files are stored. 
csv_directory = outdir
# This is read all the .csv files if name starts with `nearest_points` and ends with `compare_field` value from processing parameters.
nearest_hotspots_csv_files = [
    fp for fp in csv_directory.iterdir()
    if (fp.name.startswith("nearest_points"))
    and (fp.name.endswith(f"{processing_parameters['compare_field']}.csv"))
]

In [8]:
# nearest points csv files that will be used analysis from here on.
nearest_hotspots_csv_files

[PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_NOAA 20_VIIRS_NASA2.0NRT_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SENTINEL_3B_SLSTR_EUMETSAT_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SUOMI NPP_VIIRS_LANDGATE_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_AQUA_MODIS_LANDGATE_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SENTINEL_3A_SLSTR_EUMETSAT_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SENTINEL_3A_SLSTR_ESA_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SUOMI NPP_VIIRS_NASA1_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_AQUA_MODIS_NASA6.03_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_TERRA_MODIS_LANDGATE_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/

In [9]:
# returns a dask DataFrame with index set at column `compare_field` from processing_parameters.
nearest_points_ddf = util.csv_to_dataframe(nearest_hotspots_csv_files, processing_parameters["compare_field"])

In [10]:
nearest_points_ddf.head()

,Unnamed: 0,latitude,longitude,satellite,sensor,confidence,power,datetime,satellite_sensor_product,geometry,...,2_power,2_datetime,2_solar_day,2_satellite_sensor_product,2_geometry,2_solar_night,dist,dist_m,timedelta,count
solar_day,,,,,,,,,,,,,,,,,,,,,
2020-04-21 00:09:47,0,-27.27521,147.94824,NOAA 20,VIIRS,50.0,2.0,2020-04-20 14:18:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.94824 -27.27521),...,2.00,2020-04-20 14:18:00,2020-04-21 00:09:47,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.94824 -27.27521),2020-04-20 12:09:47,0.000000,0.000000,00:00:00,1
2020-04-21 00:09:47,19595,-27.27521,147.94824,NOAA 20,VIIRS,50.0,2.0,2020-04-20 14:18:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.94824 -27.27521),...,0.26,2020-04-21 13:08:35,2020-04-21 22:58:29,SENTINEL_3A_SLSTR_ESA,POINT (147.4761716276585 -33.87393291447654),2020-04-21 10:58:29,6.615587,732949.818254,22:50:35,1
2020-04-21 00:09:47,22552,-27.27521,147.94824,NOAA 20,VIIRS,50.0,2.0,2020-04-20 14:18:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.94824 -27.27521),...,0.26,2020-04-21 13:08:35,2020-04-21 22:58:29,SENTINEL_3A_SLSTR_EUMETSAT,POINT (147.4761716276585 -33.87393291447654),2020-04-21 10:58:29,6.615587,732949.818254,22:50:35,1
2020-04-21 00:09:47,26310,-27.27521,147.94824,NOAA 20,VIIRS,50.0,2.0,2020-04-20 14:18:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.94824 -27.27521),...,0.47,2020-04-21 12:30:35,2020-04-21 22:22:20,SENTINEL_3B_SLSTR_EUMETSAT,POINT (147.9395736264885 -27.27608093113452),2020-04-21 10:22:20,0.008710,863.482574,22:12:35,1
2020-04-21 00:09:47,30634,-27.27521,147.94824,NOAA 20,VIIRS,50.0,2.0,2020-04-20 14:18:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.94824 -27.27521),...,-1.00,2020-04-20 15:07:00,2020-04-21 00:58:48,SUOMI NPP_VIIRS_LANDGATE,POINT (147.95188 -27.27688),2020-04-20 12:58:48,0.004005,421.824027,00:49:00,1


# Results
## Co-occurrence metrics

In [11]:
region_alias = "nsw"
output_directory = processing_parameters["outdir"]
comparison_prefix = (
    f"{processing_parameters['compare_field']}"
    f"_{processing_parameters['start_date'].replace('-', '')}"
    f"_{processing_parameters['end_date'].replace('-', '')}"
    f"_{region_alias}"
)

In [12]:
# set the nearest distance threshold between two hotspots to confine the analysis within the distance threshold.  
dist_threshold = 5000  # units in meters

In [13]:
nearest_ddf_dist_subset = client.persist(nearest_points_ddf[nearest_points_ddf["dist_m"] < dist_threshold])

In [14]:
# Count of hotspot matches < dist_threshold
numerator = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="count",
    aggfunc="count"
).compute()

In [15]:
numerator.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_{dist_threshold}.csv"))
numerator.astype(int)

satellite_sensor_product,NOAA 20_VIIRS_NASA2.0NRT,AQUA_MODIS_LANDGATE,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,TERRA_MODIS_LANDGATE,TERRA_MODIS_NASA6.03,SENTINEL_3B_SLSTR_ESA,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_ESA,SENTINEL_3A_SLSTR_EUMETSAT,AQUA_MODIS_NASA6.03
2_satellite_sensor_product,,,,,,,,,,,,
NOAA 20_VIIRS_NASA2.0NRT,11584,805,21201,2528,22348,600,82,279,714,160,623,62
SENTINEL_3B_SLSTR_EUMETSAT,1500,238,2713,521,2537,356,53,801,2002,132,392,15
SUOMI NPP_VIIRS_LANDGATE,7888,760,26217,2591,16610,538,71,195,528,145,520,65
NOAA 20_VIIRS_LANDGATE,9591,739,19293,2037,23477,492,54,224,563,122,538,51
TERRA_MODIS_LANDGATE,1344,290,3421,383,2398,1394,163,260,455,182,568,16
TERRA_MODIS_NASA6.03,335,90,862,197,523,419,177,166,166,108,112,13
SUOMI NPP_VIIRS_NASA1,2426,327,6191,3186,4316,220,58,246,246,153,157,51
SENTINEL_3B_SLSTR_ESA,588,140,922,525,1050,192,53,809,802,130,129,15
AQUA_MODIS_LANDGATE,1969,817,5094,691,3831,215,32,52,162,37,86,63


In [16]:
# Count of hotspot matches - total  
denominator = util.dask_pivot_table(
        nearest_points_ddf,
        index="2_satellite_sensor_product",
        column="satellite_sensor_product",
        values="count",
        aggfunc="count",
    ).compute()

In [17]:
denominator.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_count.csv"))
denominator.astype(int)

satellite_sensor_product,NOAA 20_VIIRS_NASA2.0NRT,AQUA_MODIS_LANDGATE,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,TERRA_MODIS_LANDGATE,TERRA_MODIS_NASA6.03,SENTINEL_3B_SLSTR_ESA,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_ESA,AQUA_MODIS_NASA6.03
2_satellite_sensor_product,,,,,,,,,,,,
NOAA 20_VIIRS_NASA2.0NRT,11584,817,26213,3186,23477,1394,177,809,2002,1975,579,65
SENTINEL_3A_SLSTR_ESA,1310,131,2632,1447,1972,236,61,441,444,595,579,21
SENTINEL_3A_SLSTR_EUMETSAT,3758,316,7866,1447,6392,536,61,441,1101,1975,579,21
SENTINEL_3B_SLSTR_EUMETSAT,4324,409,7805,1603,7859,480,67,809,2002,660,203,29
SUOMI NPP_VIIRS_LANDGATE,10869,797,26217,3160,22944,1377,176,800,1822,1796,577,65
NOAA 20_VIIRS_LANDGATE,10933,814,25906,3178,23477,1358,175,809,1884,1876,579,65
TERRA_MODIS_LANDGATE,8356,760,20551,2736,16922,1394,175,622,1373,1591,525,64
TERRA_MODIS_NASA6.03,2840,355,7137,2367,5287,585,177,521,530,455,438,58
SUOMI NPP_VIIRS_NASA1,3097,331,6583,3186,5559,478,121,809,816,595,579,51


In [ ]:
# Difference of matched points closer than 5000m
difference = denominator - numerator

In [ ]:
difference.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_count_difference.csv"))
difference.astype(int)

In [ ]:
# Percentage of matched points closer than dist_threshold
percentage = (numerator / denominator) * 100
percentage = np.round(percentage, 2)

In [ ]:
percentage.to_csv(output_directory.joinpath(f"{comparison_prefix}_percentage.csv"))
percentage

In [ ]:
# Maximum time between match points < dist_threshold
timemax = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.max}
    
)

In [ ]:
timemax.to_csv(output_directory.joinpath(f"{comparison_prefix}_max_time_matched_points.csv"))
timemax

In [ ]:
# Minimum time between match points < dist_threshold
timemin = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.min}
    
)

In [ ]:
timemin.to_csv(output_directory.joinpath(f"{comparison_prefix}_min_time_matched_points.csv"))
timemin

In [ ]:
# Average distance (m) between matched points < dist_threshold
averagedist = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="dist_m",
    aggfunc="mean",
).compute()

In [ ]:
averagedist = np.round(averagedist, 2)
averagedist.to_csv(output_directory.joinpath(f"{comparison_prefix}_average_distance_{dist_threshold}m.csv"))
averagedist

In [ ]:
client.close() # close dask.distributed client